# Image backbone pre-training

Image backbone pretraining is employed in an attempt to make the whole network ensemble training faster.

First, make the required imports:

In [6]:
import sys
sys.path.append("..")
from multitudinous.utils.model_builder import build_img_backbone

The image backbone is chosen from a pre-defined list which is:
- `resnet50`: vanilla ResNet-50 with 4 input channels
- `se_resnet50`: ResNet-50 with 4 input channels and "Squeeze-and-Excitation" (SE) attention
- `cbam_resnet50`: ResNet-50 with 4 input channels and "Convolutional Block Attention Module" (CBAM) attention

Build the image backbone with the specified configuration. Optionally, you can provide a path to a weights file as the second argument if finetuning.

In [7]:
img_backbone = build_img_backbone('se_resnet50')
print(img_backbone)

ResNet(
  (conv1): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): SEBottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1